In [3]:
#1. 모듈 호출
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd 
import random
import os  
from selenium.webdriver.common.keys import Keys

In [4]:
#2. 사용자 인터페이스 만들기 
print('='*100)
print('유튜브 댓글 크롤러(대댓 제외)')
print('='*100)
key_word = input('1.유튜브에서 검색할 주제 키워드를 입력하세요(예:김어준의 뉴스공장) : ')
cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇 건 인가요? : '))
d_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇 건 입니까? : '))
f_dir = input('4.크롤링 결과를 저장할 폴더명만 쓰세요 : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+key_word)
os.chdir(f_dir+s+'-'+key_word)

f_name = s+'-'+key_word+'.txt'
fc_name = s+'-'+key_word+'.csv'
fx_name = s+'-'+key_word+'.xlsx'

s_time = time.time()

유튜브 댓글 크롤러(대댓 제외)
1.유튜브에서 검색할 주제 키워드를 입력하세요(예:김어준의 뉴스공장) : 다꾸
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇 건 인가요? : 30
3.각 동영상에서 추출할 댓글은 몇 건 입니까? : 100
4.크롤링 결과를 저장할 폴더명만 쓰세요 : 


In [5]:
#3. 웹드라이버 호출 & 유튜브 접속
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver88'
options=webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')

driver = webdriver.Chrome(path, options=options)
driver.get('https://www.youtube.com/')
time.sleep(random.randrange(2, 5))

In [6]:
#4. 유튜브 검색창에 키워드 넣고 검색실행
element1 = driver.find_element_by_name('search_query')
element1.send_keys(key_word)
element1.send_keys('\n')
time.sleep(3)

In [7]:
#5. 링크 리스트 만들기 
link1 = []
no22 = 2
no11 = 1
if cnt <= 19 : 
    html0 = driver.page_source
    soup0 = BeautifulSoup(html0, 'html.parser')
    list0 = soup0.find_all('ytd-video-renderer', class_='style-scope ytd-item-section-renderer')
    time.sleep(3)
elif cnt > 19 : 
    #무한 스크롤 걸기
    while no22 == 2 : 
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        html0 = driver.page_source
        soup0 = BeautifulSoup(html0, 'html.parser')
        list0 = soup0.find_all('ytd-video-renderer', class_='style-scope ytd-item-section-renderer')
        time.sleep(2)
        if len(list0) >= cnt : 
            break

for a in list0 : 
    href = a.find('a', id='video-title')['href']
    link1.append(href)
    no11 += 1
    if no11 > cnt : 
        break
        
driver.quit()


In [8]:
#6. 영상 하나하나 들어가서 댓글 수집하기 
f = open(f_name, 'a', encoding='utf-8')
name1 = []
date1 = []
content1 = []
url1 = []
gogam1 = []
bgogam1 = []

a = 0
no44 = 1
no55 = 1

for b in link1 : 
    path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver88'
    options=webdriver.ChromeOptions()
    options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

    driver = webdriver.Chrome(path, options=options)
    driver.get('https://www.youtube.com'+b)
    time.sleep(random.randrange(2, 5))
    
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN) #댓글 20개 발견
    time.sleep(8)
    html1 = driver.page_source 
    soup1 = BeautifulSoup(html1, 'html.parser')
    
    


    if d_cnt <= 20 : 
        dd_list = soup1.find('div', id='contents').find_all('ytd-comment-renderer', class_='style-scope ytd-comment-thread-renderer')
    
    elif d_cnt > 20 : 
        while no22 == 2 : 
            for z in range(1, 6) : 
                driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
                time.sleep(5)
            html5 = driver.page_source
            soup5 = BeautifulSoup(html5, 'html.parser')
            dd_list = soup5.find('div', id='contents').find_all('ytd-comment-renderer', class_='style-scope ytd-comment-thread-renderer')
            time.sleep(2)
            if len(dd_list) >= d_cnt : 
                break
            elif a == len(dd_list) : 
                print('%s 개 댓글 조회를 요청하셨으나, %s 개 밖에 없어서 %s 개 만큼만 조회하겠습니다' %(d_cnt, len(dd_list), len(dd_list)))
                break
            a = len(dd_list)
            

    for c in dd_list : #하나의 댓글 안에서     
        content = c.find('yt-formatted-string', id='content-text').get_text().replace('\n', "").strip()
        content1.append(content)
        print('4.댓글 내용 :',content)
        f.write(content+'\n')

        no55 += 1
        if no55 > d_cnt : 
            break

            
    no44 += 1
    no55 = 1
    driver.quit()
f.close()
e_time = time.time()
total_time = e_time - s_time 

100 개 댓글 조회를 요청하셨으나, 19 개 밖에 없어서 19 개 만큼만 조회하겠습니다
4.댓글 내용 : Omggg how cute!!!
4.댓글 내용 : 💛💙💛불루레몬에이드 같은 조합👍(?)💙💛💙
4.댓글 내용 : 슬이님 ㅠㅠ 오늘도 다꾸 넘무 예뽀요 진짜루 표꾸 장인 .. 다꾸 장인 ..💛
4.댓글 내용 : 저 또 왔어용!ㅎㅎ 저 진짜 플레인을 살지 글리터를 살지 고민이네요ㅠㅠ
4.댓글 내용 : 이거 보고 멜펙토리 빅 스티커들 싹 시켰어요😍
4.댓글 내용 : 일등이 두분이시네요💗♥️🎀🌈💖🎶💕대단요😀😃😄👍선댓글 후 영상요댓글쓰는데 넘 기대됩니다🍎🍓🍒
4.댓글 내용 : 자기 전 보는 슬이님 다꾸 영상💛💙 표꾸 잘 보고가요👍🏻🥰👏🏻속지 찢어질 때 저도 같이 흠칫! 저는 스티커 매일 찢....ㅠ
4.댓글 내용 : 큰 사이즈의 스티커도 넘 귀엽고 예쁘네요 ㅠ_ㅠ! 이번 표지도 넘넘 예뻐요! 제 스타일이에오...❤
4.댓글 내용 : 새로운 표지~💙색감 대박이예요 ❄늘 그렇듯이 😍
4.댓글 내용 : 멜리팩토리 스티커 완전 빅사이즈여서 귀엽네요-목소리 나오는 영상이 더 좋아여 :)
4.댓글 내용 : 헐 너무너무 예뻐요ㄷㄷ!! 갠적으로  색감도 디자인도 이번 표지가 젤 맘에들어요!!   그리고 슬님한테  영업당해서 정사각다욜 드디어샀습니다ㅋㅋ 오늘이 3일차에요!!ㅋㅋㅋㅋㅋ  고민하다 6공말고 실제본으로 샀지만ㅎ   그래도 정사각다이어리 진짜 넘 귀엽고 예쁘고 맘에들어요😍😍😍
4.댓글 내용 : 완전 이뻐요 !!
4.댓글 내용 : Always stunning. And your nail so pretty 💖💖
4.댓글 내용 : 슬이님 항상 다꾸도잘하고 표지도 너무 잘꾸며요~~♥♥다음영상 기대할께요~~♡♡♡
4.댓글 내용 : 퇴근하고 집에와서 자기전에 잘봤습니다 역시슬이다꾸 최고 ~  너무이쁘고~ 굿입니다 ㅋ 굿밤되세요~^^
4.댓글 내용 : 알람 뜨자마자 왔는데 표지 꾸미기 어떻게 저렇게 잘 하시는지..ㅜ.ㅜ진짜 이뿌게꾸미세여..부럽습니닷♡희희 영상 너무이쁘고 전 슬이님 최근

4.댓글 내용 : 꾸미는것도 꾸미는거지만 작은 글씨로 예쁘게 반듯하고 일정하게 글씨쓰시는게 진짜 대박이시다..
4.댓글 내용 : 저렇게 모아놓으면 뿌듯하고 이쁘네요.... 너무 부럽습니다... 저렇게 이쁘게 꾸미실줄 아시니..
4.댓글 내용 : 와 저렇게 꾸미려면 스티커 값이....
4.댓글 내용 : 헐헐 대박진짜 이뻐요...오늘 다꾸 시작했는데..!역시 차원이 다르다 글도 전 2줄만 쓰고 ㅋㅋ 다이어리 꾸미기 방식 조금 참고해도 될까영??
4.댓글 내용 : 무작정 스티커 많이 붙이고 꾸미는 게 제일 예쁜 줄 알았는데 초반다꾸 진짜 깔끔한데 예쁘네요 .. ㅜㅜ 저도 덕지덕지 붙이는 것 좀 줄여봐야겠아요 !!
4.댓글 내용 : 이정도가 이상하면 저는 ........하하!!
4.댓글 내용 : 저도 가끔 꾸미기만 해놓고 일기쓰는거 까먹는 날이 있는데 그러면 나중에 일기 대신 노래가사를 써요..ㅋㅋ 비어있는게 싫어서 써본건데 괜찮더라구요!!
4.댓글 내용 : So lovely!!!! It’s a pity I don’t know Korean😭
4.댓글 내용 : 지예리님 안녕하세요오~~!!처음 놀러와봤어요.^^다꾸 정말 잘 하세요....ㅠㅠ영상 보고 바로 구독이랑 좋아요 눌렀어요♡♡♡(자랑)앞으로 아직 안 본 영상도 열심히 다 보고,올라오는 영상은 꼬박꼬박 볼게요~~~!!
4.댓글 내용 : 정말 이뻐요! 처음으로 다꾸하시는 유튜버 구독 눌렀어여❤ 제 다꾸러 우상 이세염🧡 ( 내 마음 속 1픽...👍 )
4.댓글 내용 : 진짜 너무 예뻐요옴 ㅠㅠ 죄송하지만 보라색 다이어리 어디서 사셨는지 이름이랑 구매처 알 수 있을 까욤 ㅠㅠ 저런 스타일 좋아하는뎀..!🥺❤️
4.댓글 내용 : 글씨 진짜 잘쓴다..
4.댓글 내용 : 글씨 진짜 잘쓴다
4.댓글 내용 : cute♡♡♡
4.댓글 내용 : 걍 감탄밖에ㅜ안나오네요....
4.댓글 내용 : 저번에 댓글에 답글 남겨줘서 감사합니다 ㅎㅎ 오늘 빨리 왔어요!!  다이어리를 이렇게 잘꾸미다니!  제에리님이 꾸민거 이렇게 한번에 다 볼수있어

4.댓글 내용 : 이ㅣ쁘게 쟈 ㄹ해따ㅏㅏ🤍
4.댓글 내용 : 와... 못하는게 대체 뭐닝...
4.댓글 내용 : 제품들은 다이소가 좋지만 꾸미신거는 아트박스 게 더 이뿌게 잘꾸미신것 같아용❤️
4.댓글 내용 : 저는 아트박스꺼가 더 귀엽다고 생각해여
4.댓글 내용 : 옐언니는 툭툭 던지는것 같은데 되게 잘하는느낌...?
4.댓글 내용 : 야 너 잘한다? 다음에 같이해
4.댓글 내용 : 저 이 영상 보면서 같이 다꾸 했는 데 언니가 저보다 잘했어요! ㅠㅠ
4.댓글 내용 : 둘다 넘 잘해서 진짜 못 고르겠웅~!진쫘 초보 맞아유~?!
4.댓글 내용 : 언니 다꾸 아주~~~잘했어요^^♡♡언니는 너무너무 예쁜게 잘했어요저가 2학년인데요 언니라고 불러다며 죄송합니다.
4.댓글 내용 : 와 나보다 잘한드앙ㅠㅠㅠㅠㅠㅠㅠㅠ
4.댓글 내용 : 이뻐요ㅠㅠㅠ힘들지마ㅜㅜ
4.댓글 내용 : 앞으로 다꾸 계속 올려주세여ㅠㅠㅠ
4.댓글 내용 : 나도 다꾸 좋아하는뎅 ㅠㅠㅠ 같이하구 싶당 ㅠㅠㅠ
4.댓글 내용 : 와~옐온니 진쫘 완전 다꾸~와~♡너무 귀엽구 이뻥ㅠㅠ 으으으윽ㅠㅠ아트박스에 가서 당장 가야겠넹~^^♡❤😭😭 구리고 저는 아트박스 다꾸가 개인적으로 더~^^좋아엽!!
4.댓글 내용 : 언니 내가요 너무 이쁜데 글씨를 조꿈씩 써조야 더 예뻐유 엉니 소질있어
4.댓글 내용 : 언니 늦어서 미안해ㅠㅠ
4.댓글 내용 : 진짜 너무예뻐요ㅠㅠ다꾸 많이 해주셨으면 좋겠어요!!
4.댓글 내용 : 마구마구 붙이시는데 넘예 🥰 다꾸 컨텐츠 너무 좋아요ㅜㅜ 다꾸 영상 많이 올려주세요☺
4.댓글 내용 : 와~처음치고는 완죤 잘 한다!^^(감탄)
4.댓글 내용 : 저는 아트박스용!! 넘~ 예뻐용~!!🥰
4.댓글 내용 : 옐언니 너무 이뻐여ㅠㅠ
4.댓글 내용 : 옐언니 너무 귀여워욘!
4.댓글 내용 : 옐언니 너무 귀여워요💗😆
4.댓글 내용 : 꺄아💕 옐언니당🙃헉....2만원으로 저렇게 다꾸가 되나여?💦💦옐언니 금손이당,!😎🤲 !!
4.댓글 내용 : 넘 예뻐요ㅜㅜ다꾸 완전 잘하시는 데욥?!
4.댓글 내용 :

4.댓글 내용 : 진짜 절대 삭제하지 말아주세요 ㅠㅠㅠ̑̈ 내년부터 시작하려고 집에서 혼자 영상만 무지 많이 보고있어요 ㅋㅋㅋ 영상 진짜 잘보고있어요 !
4.댓글 내용 : 1:01 - 다이소 핀셋 1000￦⤷(품번 1018497)3:39 - 다이소 도트타입 풀테이프 1000￦⤷(품번 1002675)7:35 - PILOT JULCE UP 펜 0.3/0.4 판매처에따라 가격 다름10:03 - 다이소 원형견출지 1000￦           ⤷(품번 다양)+위너디스 하트스티커12:48 - 휴대용 재단기 3000￦ ⤷(품번 1005138)15:16 - 러브미모어 체크패턴 메모지 믹스팩 15가지 각 15~20장 10000￦17:04 - 다이소 4단다용도 서납형 케이스 5000￦ ⤷(품번 1013300)시간없는데 빨리빨리 참고하고 원하는거 보고싶다 하시는분들?있으실거같아서 조금이나마 도움될까 해서 [시간-물품명-가격-품번(있는경우)]으로 정리했습니다
4.댓글 내용 : 15:24 맨 위에 있는 고방 체크 떡메 네이버에 검색 뭐라고 해서 나왔나요??
4.댓글 내용 : 지에리님! 혹시 파일럿 쥬시스? 펜 물번짐은 없나요??
4.댓글 내용 : 너무 좋은영상 ♥️ 진짜 도움이 많이 됬어요 진짜 지에리님 라뷰해요 💓
4.댓글 내용 : 난 그런줄 몰르고 일반 투명그걸로 사는데...ㅠ
4.댓글 내용 : 찐~~~~~~~~~~~~~~~~짜 꿀템들이네용!!!!지에리님의 완성된 다꾸만큼 너무ㅜ 이뻐요ㅠㅜㅠㅜㅠㅠ(스티커가!!!>.<)오늘두 좋은영상 감사합니당~~~!>.<(15:17 아이디어(?) 가 넘 귀엽구 웃기자나요ㅠㅜㅠㅎㅎ)
4.댓글 내용 : 너모 힐링되는 영상이에오,,♡ 목소리두 넘 죠아요ㅠㅠ
4.댓글 내용 : 2분전!! 저 바로 삽니다
4.댓글 내용 : 저 지에리님 팬이에요^^마침 지금 다꾸하고 있는뒝♡
4.댓글 내용 : 우왕!목소리도 넘 좋으시고💗설명도 넘 잘하시는 것 같아요ㅠ요즘 다꾸할려구ㅠ 해서 찾아봤는데 우연히 지에리님 영상보고 요즘너무너무 잘 보고있어요ㅠ💗🥰앞으로도 건강하시고!

100 개 댓글 조회를 요청하셨으나, 9 개 밖에 없어서 9 개 만큼만 조회하겠습니다
4.댓글 내용 : 목소리 너무 귀여우세요😎편집도 깔끔하고 보기 조아요!
4.댓글 내용 : 엇 7:30 망고베어 아닌가요..?
4.댓글 내용 : 훈구 축하드려요!☺️
4.댓글 내용 : 너무 잘 보고 가요!
4.댓글 내용 : 7:29 여기 상점 혹시 어딘지 알수있을까요?(제가 인스타가 없어서ㅠㅜ 혹시 대댓으로 알려주실수 없을까요?ㅠㅠ)
4.댓글 내용 : 네이버에 엘레강스쩡 검색하면 스티커 만원 랜덤팩 있는데 그거 진짜 혜자에료~~ 그거 구매하세요!! 베어팩도 있어욤
4.댓글 내용 : 다음엔 언박싱한 걸로 다꾸하는 것도 보여주세요!! >_<
4.댓글 내용 : 랜덤팩오천원짜리상점알려주세요
4.댓글 내용 : 몇살이세요??
100 개 댓글 조회를 요청하셨으나, 31 개 밖에 없어서 31 개 만큼만 조회하겠습니다
4.댓글 내용 : 와웅!!ㅋㅋ저도 2일치 다꾸 밀렸는데 같이 써서 재밌게 봤네용!!ㅋ (78빠 쿜)
4.댓글 내용 : 정말 금손이세요ㅠㅠㅠㅠㅠ가끔 보는데 볼 때마다 힐링중이예용
4.댓글 내용 : 와 이십그램님 진짜 금손이시네요..❤💛
4.댓글 내용 : 글씨가 정말 이뻐요~^^💕저는 말하면서 다꾸 하기가 어렵던데 이십그램님은 말하면서 마테도 쓱싹 잘 붙이시공 색감도 이쁘게 잘 고르시네용~😊
4.댓글 내용 : ㅠㅠ너무 귀엽고 깔끔하게 잘꾸미 세요ㅠ 오늘도 배워갑니당!
4.댓글 내용 : 와ㅠㅠ 역시 이십그램님이시네요👍👍ㅠㅠ오늘도 잘 보고 갑니다~! 좋은 영상 감사드려요❤❤😍
4.댓글 내용 : 헐 감사합니당 정신없어서 다꾸 못했는데 ㅠㅠ
4.댓글 내용 : 마테로 완전 작품을 만드셨네요..!저도 떡메모지 그냥 네모모양 그대로 붙이기만 하는건 예쁘게 꾸미기도 어렵고 싫었는데 이 영상 보면서 하면 이쁘게 할수있을거같아요..!!🥰❣
4.댓글 내용 : 마테 활용 진짜 잘하시는 것 같아요~볼때마다 감탄하며 본답니다^^ 엄청 많은 참고가 되고 있어요~~ㅎㅎ
4.댓글 내용 : 펜 정보 알 수 있을까요

100 개 댓글 조회를 요청하셨으나, 29 개 밖에 없어서 29 개 만큼만 조회하겠습니다
4.댓글 내용 : 꺄휴! 1등!! 저희 다이소에 없어서.. 진짜 몇번을 가봤는데 없더라구요..ㅠㅠ 헤다님 구하셨다니 넘넘 축하하고 부럽습니당
4.댓글 내용 : 저는 그 진한 알스, 숫스를구했는데 하나밖에 안 남아있어서하나밖에 못샀어요ㅜㅠ그리고 더 사고싶어서 여러번 다이소에 들렸는데 다 팔렸는지 없더라구요ㅜㅠㅠ
4.댓글 내용 : 금테 컨페티도 새로 나왔어요!!! 그건 대체로 밀림 없이 퀄리티가 조금 더 괜찮더라구요~~~ 품번은 1029985니까 한 번 사보셔도 좋을 것 같아요!!
4.댓글 내용 : 중간에 다이소 숫자 스티커랑 알파벳 스티커 진짜 가성비👍🏻👍🏻👍🏻👍🏻👍🏻
4.댓글 내용 : 선댓 후 감상합니다♥  오늘도 좋은 영상 감사해요
4.댓글 내용 : ㅠ.ㅜ 택배로 알스 수스 모스 컨스 사야 될 것 같아요 😅나도 사고 싶다🙏🏻
4.댓글 내용 : ㅠㅠㅠ오늘 명동 11층 다이소에도 다녀왔는데 없더라구요 도대체 언제야 제 수중에 떨어질 수 있을지...ㅎ 다이소 더 열일하자~~~!!
4.댓글 내용 : 우왕 제가 씰스티커 사러 갔는데 없네용 ㅠㅠ 부럽다!
4.댓글 내용 : 휴.... 부산에는 없어여 ㅠㅠ
4.댓글 내용 : 저두요 오늘 네군대나 돌아다녔는데 ㅠㅠ 없다네요 ㅠㅠ
4.댓글 내용 : 알파벳스티커는  첨보는데  와~~ 갖고 싶어요😳😳 다이소  영상 때문에  최근에  방문한적이  있는데  난   왜 못봤지  ㅠㅠ
4.댓글 내용 : 저희 다이소는 2층인데 작아서 없어요ㅠ
4.댓글 내용 : 알스나 숫스,컨페티 용인 동백 다이소에 남아돌아요!
4.댓글 내용 : 푸 마스킹 테이프를 반대로 끼워서 뜯으면 더 잘 뜯어질 것 같아요 !!
4.댓글 내용 : 유튜버분 딱 내스탈 품번도 같이 말씀해주시는 센스 어떻게 사용할지도 이야기해주면 나도 저렇게 사용해보고 싶다는 생각드는 영상임
4.댓글 내용 : 저도 다이소 3곳을 돌아다녔었는데 아직도 진한 컨페티랑 숫자스티커는 아예 못뵜어요ㅠㅠㅠ
4.

100 개 댓글 조회를 요청하셨으나, 43 개 밖에 없어서 43 개 만큼만 조회하겠습니다
4.댓글 내용 : 영상품질을 1080p로 높여주세요.🙃안녕하세요! 잘 지내고 계신가요?조용한 다꾸 영상 들고 왔습니다.^,^저번까지는 계속 데일리 플래너 꾸미기를 보여드렸는데 오늘은 일기장이에요.영어 필기체에도 관심이 많으셔서조만간 필기체에 관한 영상도 들고 오려고 합니다.모두 편안히 감상하시고,구독과 좋아요, 알림설정으로다음 영상을 편히 받아보세요😄😉다이어리: 미도리 트래블러스 노트 오리지널 카멜BGM : (Epidemic Sound 스트리밍 저작권 구독)      Fractured Memories - Gavin Luke이메일: chadahye@gmail.com인스타: cha.dahye
4.댓글 내용 : 뜬거보자마자 바로 보러왓어용 아니어떻게이렇게감각적으로 꾸미시나요... 진짜너무신기해요... 저언니영상보고 다꾸충동 엄청 생겨서  할까말까 맨날 고민하다 워낙똥손이라 그냥 보는걸로 만족하려구요ㅋㅋㅜㅜㅜㅜ큭 그러니까 영상 자주올려주세용!😍😍😍
4.댓글 내용 : 목소리가 너무 좋으셔서 다꾸하실 때 더빙하는 영상도 자주 올라왔으면 좋겠어요ㅠㅠ
4.댓글 내용 : 영상너무 힐링되요^^ 살짝 asmr같기도 하고 영어필기체도 너무 멋있어요:)
4.댓글 내용 : 자주 자주 영상 올려주세요~^^ 매일 기다린답니다^^ 마음이 차분해지는 느낌이예요~ 얼굴도 이쁜 분이 목소리도 이쁘고 말도 잘하고 글씨도 이쁘고 꾸미기도 잘하고~ 늘 정돈된 느낌이 좋네요^^
4.댓글 내용 : 차차님 영어 쓰기하는거 따라해보겟다구한영일기 써봤는데 3일만에 포기해써요 ㅎ
4.댓글 내용 : 오늘 알림 울리자마자 달려왔네요^^ㅎㅎ 오늘도 잘 보고 갈께요^^
4.댓글 내용 : 차차님 만의 컨텐츠 너무 좋네용😊😊 항상 열심히 볼게욤
4.댓글 내용 : 또각또각 슥슥 ㅠㅠ 맘이 너무 힐링되고 차분해져요 👏🏻 구독하고갑니당 차차님 !
4.댓글 내용 : 언니 진짜 완벽한 캐릭터 아닌가요ㅜㅜ 못하는게 뭐에요!? 제발 알려줘봐요 ㅋ

4.댓글 내용 : 범지님 다음번에는 asmr 해주세요ㅠㅠ목소리가 너무 소곤손곤 말하시는거 같아요!!!그래서 범지님을 좋아하는 이유 중 하나입니다!!!
4.댓글 내용 : 2:51 먼지 궁금해서 제꺼 바인더 펼쳐봤더닠ㅋㅋㅋㅋㅋ꽃다발이었어요 두송이밖에 없긴하지만ㅎ💐🌷🌹
4.댓글 내용 : 범❤범지❤범지님❤범지님사❤범지님사랑❤범지님사랑해❤범지님 사랑해요❤범지님사랑해❤범지님사랑❤범지님사❤범지님❤범지❤ 범지님 저진짜 기분 너무 좋아요. 오늘 하루 피로를 범지님 영상을 보고힐링하며 맨날 기분이 좋게 잠듬니다 진짜루 범지님 목소리 쵝오😆👍범지님 안녕히 주무세요❤ 😊
4.댓글 내용 : 꺄아 안냐세유 범지님 선댓후감상인데 인트로보니까 짱이뻐요ㅠㅠ 다꾸 진짜 잘하세요ㅠ😍
4.댓글 내용 : 범지님 ㅠㅠㅠㅠㅠㅠ(´°̥̥̥̥̥̥̥̥ω°̥̥̥̥̥̥̥̥｀)이게 세계일 귀려운 것 같아요❤️
4.댓글 내용 : 2:51 짤주머니같아요ㅋㅋㅋㅋㅎㅎㅎㅎㅋㅋㅋㅋㅋ
4.댓글 내용 : 범지님 하는거보구...다꾸 입덕...너무재밌씁니댜..
4.댓글 내용 : 좀...늦게 왔다..ㅠㅠ
4.댓글 내용 : 진짜 너무 예뻐요 범지님 존중하고 사랑해요
4.댓글 내용 : 최고예요~~
4.댓글 내용 : 와우!!! 65등!! 범지님 영상 와서 행벅해유!!~^^
4.댓글 내용 : 좀 늦었네요 ㅠㅠ 잘보고갈께요💗💗 굿밤되세요 범지님🥰
4.댓글 내용 : 스티커 빼면 시체인(?) 범지님!!스티커 계속 추가하시는 범지님 너무 귀여우세요..ㅎㅎ
4.댓글 내용 : 저 얼마전에 구독했어요!!범지님 보고 저도 스꾸하려고 스티커 주문했습니당ㅋ목소리너무 좋아요~^^그리구 범지님 너무 잘 꾸미네요♡
4.댓글 내용 : 진짜 처음으로 일찍 왔어요..ㅠㅠ  다꾸 완전 잘하셔요😍 넘 이쁨니다ㅠㅠ
4.댓글 내용 : 범지님 목소리 너무 귀여운거 아니에요??ㅠㅠ심장폭행 하지마요!!!ㅠ
4.댓글 내용 : 오일파스텔로 그림 그리다가 달려왔어요 ㅎㅎㅎ매일 재미있는 영상 올려주셔셔 감사해요😆
4.댓글 내용 : 범지님은 뭘해도 다 잘하시는것 같아요...ㅜ💞
4.댓글

100 개 댓글 조회를 요청하셨으나, 3 개 밖에 없어서 3 개 만큼만 조회하겠습니다
4.댓글 내용 : 앵쏘님 다꾸 너무 잘하세요~~♥♥앵쏘님 영상보고 다꾸배워갑니다~~!!💕다음영상 기대할께요~~♥♥💕💝
4.댓글 내용 : 앵쏘님 다이어리 너무 이뽀요^^ 마자요!!앵쏘님 다꾸배치 금손이셔용👍👍👍 앵쏘님의 다꾸하는 방법이라니😲😲 꼭 도전해보겠습니다~ㅎㅎ 팁알려주셔서 감사해요❤❤ 펜트하우스 너무 재밌어요!!펜트하우스 2도 나온다고하니까 기대되고 빨리 보고싶네요😭😭 앵쏘님 다음영상도 기대되는데용>< 오늘도 이쁜영상 잘보구 갑니당:)
4.댓글 내용 : 전  잘 찍는 방법두 알고 싶습니다..앵쏘님 인스타를 보면 다이어리 사진도 너무 이쁘게 잘 찍으시더라구요😭역시 앵쏘님은 금손이심미다(*´ ˘ `*)다꾸 팁 다 쓸어담아 갑니당:O스티커로 꽉찬 다꾸 진짜 귀엽구 예뻐요💙다꾸 팁을 주셨으니 주접 조금만 부리고 갈게요앵쏘님 저 작년부터 그늘을 못보게 되었어요앵쏘님만 보면 빛이나서 저희 집에 불을 킬일이 없어요 그래서 전기세가 안나와서 한국전력공사에서 저희집에 찾아왔었다니까요?그리고 앵쏘님 영상 볼때 너무 좋아서 이마를 쳤더니 거북목이 해결 됐어요앵쏘님은 절대 경마장에 가면 안되겠어요.앵쏘님을 보면 말이 안나오니까..여기까지만 하고 갈게요 앵쏘님 영상 잘봤습니다💜
100 개 댓글 조회를 요청하셨으나, 58 개 밖에 없어서 58 개 만큼만 조회하겠습니다
4.댓글 내용 : 이 채널 뭔가 떡상각이다
4.댓글 내용 : 저 방에서 살면 무슨 기분일까..ㅠ
4.댓글 내용 : I don't understand you, and I was a very nice video. 🥺❤️❤️
4.댓글 내용 : 진짜 영상 너무 이뻐요ㅠㅠ❤️ 많은 다꾸 유튜브 보는데 캬야님 영상이 제일 깔끔하고 목소리도 좋고 너무 즐겨봐요,,ㅎㅎ💕 봐도봐도 질리지 않고 재밌어요ㅋㅋㅋㅋ😆
4.댓글 내용 : 왕!!ㅜㅜ 영상 아껴 볼거에요♡♡♡
4.댓글 내용 : 커팅매트 작은 거 찾으시면 다이소 커팅매트도 조심스레 추천해보아용!

4.댓글 내용 : 무엇보다 저많은 스티커들과 메모지들의 출처를 알고 있다는게 신기해요ㅋㅋㅋ
4.댓글 내용 : 보쨘님: 자 다음 거 보여드릴게요나: 우와.. 엄청 잘하셧다.. 역시 보쨘님..보쨘님: Aㅏ.. 이거 좀 망했네요.나: ..?????
4.댓글 내용 : 보쨘님 대학생인줄 알았는데  벌써 결혼까지 하셨다니 ㄷㄷ
4.댓글 내용 : 나 : 스티커 어디제품이지 개이쁘다...보쨘님: 이건 제가 그린거에요나 : ?나: 와 저거는 엽서 붙인건가보쨘님: 행성이나 별을 그린거에요나: ?
4.댓글 내용 : 오늘 이걸본 나의 말 "2020년부터해야지"수정후"2021년부터 해야지"
4.댓글 내용 : Your journal pages are so beautiful 🤗💕
4.댓글 내용 : 타인의 이런 엄청난 노력을 술술 엿볼수잇다는게 너무 감사하네요~~ 완전 눈호강이에요!!
4.댓글 내용 : 계속 스타일이 바뀌지만 한결같은 건 바로 정성.. 저는 글자 쓰다가도 지쳐서 글씨 날라가는데 보쨘님은 꾸준하면서도 참 정성스럽게 꾸미시는 것 같아요!  멋지십니다!!
4.댓글 내용 : 나:우와.. 저 스티커 예쁘다🥰???: 이건 제가 그린고에오나:!?!
4.댓글 내용 : 종이 넘기는 소리 완전 좋은......❤
4.댓글 내용 : 손그림이 진짜 제일 이쁜것 같아요
4.댓글 내용 : 저는 아이패드 굿노트 앱으로 다꾸하는데 보쨘님 영상들 보면 역시 아날로그식 종이 다이어리가 짱인거같아요 ㅠㅠ 디지털로는 따라가지 못하는 감성,,? 저번영상 2020 체리 다이어리 뽐뿌왔어요!!ㅎㅎ 점점 늘어가시는 실력이 보여요 앞으로도 예쁜 다꾸영상 많이 올려주세요!! (저는 보쨘님 투머치다꾸도 좋아요❣)
4.댓글 내용 : your diaries are so lovely 💖 hoping for english subtitle 😊
4.댓글 내용 : 장마다 종이가 다 다르고장마다 엄청난 시간을들여스티커와 데코가 가능하다니 ㄷㄷ
4.댓글 내용 : 다이어리 꾸미는 이런 영상들 볼때마다 뭔가 내용을 꾸민다기보다는 다이어리를 

In [ ]:
#7. csv, xlsx 파일로 저장하기 
you = pd.DataFrame()
you['URL 주소'] = url1
you['댓글작성자명'] = name1
you['댓글작성일자'] = date1
you['댓글내용'] = content1
you['공감횟수'] = gogam1
you['크리에이터 하트'] = bgogam1

you.to_csv(fc_name, encoding='utf-8-sig')
you.to_excel(fx_name)

print('='*100)
print('총 %s 개 영상의 댓글을 수집하였습니다' %(cnt))
print('총 소요된 시간은 %s 초 입니다' %round(total_time, 1))
print('='*100)


19